In [7]:
import serial
import time
import csv
import os


PORT = '/dev/ttyACM0'  # Change to your Arduino's port for Raspberry Pi (e.g., /dev/ttyUSB0 or /dev/ttyACM0)
BAUD = 9600

# Data type selection
options = [
    "Baseline Readings",
    "Soy Sauce",
    "Fish Sauce",
    "Oyster Sauce",
    "Worcestershire Sauce"
 ]
print("Select data type:")
for i, opt in enumerate(options, 1):
    print(f"{i}. {opt}")
choice = input("Enter number (1-5): ").strip()

try:
    choice = int(choice)
    assert 1 <= choice <= 5
except:
    print("Invalid selection. Exiting.")
    exit(1)

selected = options[choice-1]

# File and label setup
if selected == "Baseline Readings":
    filename = "baseline.csv"
    mode = "w"  # Overwrite
    trial = None
else:
    filename = selected.lower().replace(" ", "_") + ".csv"
    mode = "r+" if os.path.exists(filename) else "w+"
    trial = input("Enter trial number: ").strip()
    try:
        trial = int(trial)
        assert trial > 0
    except:
        print("Invalid trial number. Exiting.")
        exit(1)

try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
    time.sleep(2)  # Wait for Arduino to reset
except Exception as e:
    print(f"Error opening serial port {PORT}: {e}")
    exit(1)

header = ["Trial", "MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137", "Label"]
if selected == "Baseline Readings":
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header[1:])
        print(f"Recording baseline readings to {filename} for 1 minute. Press Ctrl+C to stop early.")
        start_time = time.time()
        try:
            while time.time() - start_time < 60:  
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    values = line.split(",")
                    if len(values) == 6:
                        writer.writerow(values + ["Baseline"])
                        print(values, "Baseline")
                    else:
                        print(f"Unexpected data format: {line}")
        except KeyboardInterrupt:
            print("Stopped by user.")
else:
    # Read all rows, update or append trial
    with open(filename, mode, newline="") as f:
        reader = csv.reader(f)
        rows = list(reader)
        # If file is empty, add header
        if not rows:
            rows.append(header)
        # Remove all rows for this trial
        rows = [row for row in rows if not (row and row[0] == f"Trial {trial}")]
        print(f"Recording trial {trial} for {selected} to {filename} for 10 minutes. Press Ctrl+C to stop early.")
        new_trial_rows = []
        start_time = time.time()
        try:
            while time.time() - start_time < 600:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    values = line.split(",")
                    if len(values) == 6:
                        new_row = [f"Trial {trial}"] + values + [selected]
                        new_trial_rows.append(new_row)
                        print(f"Trial {trial}", values, selected)
                    else:
                        print(f"Unexpected data format: {line}")
        except KeyboardInterrupt:
            print("Stopped by user.")
        # Write all rows back to file, with new trial data
        f.seek(0)
        f.truncate()
        writer = csv.writer(f)
        writer.writerows(rows + new_trial_rows)

ser.close()
print("Serial port closed.")

Select data type:
1. Baseline Readings
2. Soy Sauce
3. Fish Sauce
4. Oyster Sauce
5. Worcestershire Sauce
Recording trial 13 for Fish Sauce to fish_sauce.csv for 10 minutes. Press Ctrl+C to stop early.
Unexpected data format: 9,17,36,260,349
Trial 13 ['20', '70', '18', '37', '259', '347'] Fish Sauce
Trial 13 ['20', '70', '17', '37', '260', '349'] Fish Sauce
Trial 13 ['20', '70', '16', '36', '260', '351'] Fish Sauce
Trial 13 ['21', '71', '18', '36', '260', '350'] Fish Sauce
Trial 13 ['20', '71', '18', '37', '260', '349'] Fish Sauce
Trial 13 ['20', '71', '17', '36', '261', '350'] Fish Sauce
Recording trial 13 for Fish Sauce to fish_sauce.csv for 10 minutes. Press Ctrl+C to stop early.
Unexpected data format: 9,17,36,260,349
Trial 13 ['20', '70', '18', '37', '259', '347'] Fish Sauce
Trial 13 ['20', '70', '17', '37', '260', '349'] Fish Sauce
Trial 13 ['20', '70', '16', '36', '260', '351'] Fish Sauce
Trial 13 ['21', '71', '18', '36', '260', '350'] Fish Sauce
Trial 13 ['20', '71', '18', '37'

In [ ]:
import pandas as pd

# Sensor columns
sensor_cols = ["MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137"]

# Load baseline and compute averages
baseline_df = pd.read_csv("baseline.csv")
baseline_avg = baseline_df[sensor_cols].astype(float).mean()

# Sauce file names and output names
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

for sauce in sauces:
    input_file = f"{sauce}.csv"
    output_file = f"{sauce}_corrected.csv"
    try:
        df = pd.read_csv(input_file)
        # Subtract baseline average from each sensor column
        for col in sensor_cols:
            df[col] = df[col].astype(float) - baseline_avg[col]
        df.to_csv(output_file, index=False)
        print(f"Corrected file saved: {output_file}")
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

: 

In [8]:
# USE FOR SUMMARIZATION OF DATA

import pandas as pd

# Sensor columns in your desired order
sensor_cols = ["MQ2", "MQ3", "MQ135", "MQ136", "MQ137", "MQ138"]
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

summary_rows = []

for sauce in sauces:
    input_file = f"{sauce}.csv"
    try:
        df = pd.read_csv(input_file)
        # Group by trial
        for trial, group in df.groupby("Trial"):
            means = group[sensor_cols].astype(float).mean()
            row = [sauce.replace("_", " ").title()] + list(means)
            summary_rows.append(row)
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Create summary DataFrame
summary_df = pd.DataFrame(
    summary_rows,
    columns=["Label"] + sensor_cols
)

summary_df.to_csv("sauce_trials_summary_mean.csv", index=False)
print("Mean summarized trial data saved to sauce_trials_summary_mean.csv")

Mean summarized trial data saved to sauce_trials_summary_mean.csv


In [32]:
# MEAN STD MAX MIN AUC

import pandas as pd
import numpy as np

sensor_cols = ["MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137"]
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

summary_rows = []
feature_names = []
for sensor in sensor_cols:
    feature_names += [f"{sensor}_mean", f"{sensor}_std", f"{sensor}_max", f"{sensor}_min", f"{sensor}_auc"]

for sauce in sauces:
    input_file = f"{sauce}.csv"
    try:
        df = pd.read_csv(input_file)
        # Group by trial
        for trial, group in df.groupby("Trial"):
            features = []
            for sensor in sensor_cols:
                vals = group[sensor].astype(float).values
                features += [
                    np.mean(vals),
                    np.std(vals),
                    np.max(vals),
                    np.min(vals),
                    np.sum(vals)  # AUC as sum
                ]
            row = [trial] + features + [sauce.replace("_", " ").title()]
            summary_rows.append(row)
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Create summary DataFrame
summary_df = pd.DataFrame(
    summary_rows,
    columns=["Trial"] + feature_names + ["Label"]
)

summary_df.to_csv("sauce_trials_summary_advanced.csv", index=False)
print("Advanced summarized trial data saved to sauce_trials_summary_advanced.csv")

Advanced summarized trial data saved to sauce_trials_summary_advanced.csv
